In [ ]:
# Consolidate all imports to one cell at the top
from pathlib import Path
import pandas as pd
import plotly.express as px

In [ ]:
# Loading data with Pandas

# Define the correct path to the CSV file
file_path = Path("/home/charles/github/repos/learn-pandas/data/mock_data.csv")

# Load the data

data = pd.read_csv(file_path)

# Display the first 5 rows of the data
print(data.head())

In [ ]:
# Cleaning and transforming data with Pandas
# Define the correct path to the CSV file
file_path = Path("/home/charles/github/repos/learn-pandas/data/mock_data.csv")

# Check if the file exists before reading
if not file_path.exists():
    raise FileNotFoundError(f"File not found: {file_path}")

# Load the data
data = pd.read_csv(file_path)

# Convert 'Date' column to datetime and handle any parsing errors
if "Date" in data.columns:
    data["Date"] = pd.to_datetime(
        data["Date"], errors="coerce"
    )  # Coerce invalid dates to NaT
else:
    raise KeyError("'Date' column not found in the dataset.")

# Handle missing values (if any)
data = data.dropna()

# Filter data for a specific time range
filtered_data = data[
    (data["Date"] > "2019-03-22") & (data["Date"] < "2019-10-26")
].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Add a new column for Total Sales only if 'Sales' and 'Quantity' columns exist
if "Sales" in filtered_data.columns and "Quantity" in filtered_data.columns:
    filtered_data.loc[:, "Total Sales"] = (
        filtered_data["Sales"] * filtered_data["Quantity"]
    )  # Use .loc to avoid the warning
else:
    raise KeyError("One or both of the columns 'Sales' and 'Quantity' are missing.")

# Display the first few rows to verify the transformations
print(filtered_data.head())

Key Improvements:

File Existence Check: Before attempting to read the CSV file, we now check if the file exists. If not, a FileNotFoundError is raised.

Handling Date Parsing: We ensure that invalid dates (if any) are handled using errors='coerce', which converts invalid dates to NaT.

Checking for Columns: The code now checks for the existence of the 'Date', 'Sales', and 'Quantity' columns before performing operations. If a column is missing, a KeyError is raised.

Avoiding SettingWithCopyWarning: Adding the new 'Total Sales' column directly to the filtered_data DataFrame is done safely to avoid chained indexing warnings.

Debugging Output: Added print(filtered_data.head()) to see the first few rows after the transformations.

Key Fixes:
.copy(): I added .copy() when creating filtered_data. This ensures that the new DataFrame filtered_data is a full copy, not just a slice from data, which avoids the warning.

.loc[] for Assignment: I used .loc[] to explicitly modify the DataFrame by specifying the rows and columns to modify. This avoids any ambiguity and makes the operation more predictable.

Why the Warning Occurs:
The warning appears because pandas sometimes creates "views" rather than full copies when slicing DataFrames. Modifying a view can lead to unexpected behavior, so pandas raises the warning to prevent that.

In [ ]:
# Define the correct path to the CSV file
file_path = Path("/home/charles/github/repos/learn-pandas/data/mock_data.csv")

# Check if the file exists before reading
if not file_path.exists():
    raise FileNotFoundError(f"File not found: {file_path}")

# Load the data
data = pd.read_csv(file_path)

# Assuming 'Total Sales' column exists in data, if not, create it
if (
    "Total Sales" not in data.columns
    and "Sales" in data.columns
    and "Quantity" in data.columns
):
    data["Total Sales"] = data["Sales"] * data["Quantity"]

# Group the data by 'Product' and calculate the total sales for each product
product_sales = data.groupby("Product", as_index=False)["Total Sales"].sum()

# Sort the grouped data by 'Total Sales' in descending order
sorted_product_sales = product_sales.sort_values(by="Total Sales", ascending=False)

# Display the sorted results
print(sorted_product_sales)

Explanation:
groupby(): We use groupby('Product') to group the data by the 'Product' column.

Aggregation with sum(): Since groupby() needs an aggregation function, I use .sum() to sum the Total Sales for each product.

Sorting: After grouping and aggregating, the resulting DataFrame is sorted by 'Total Sales' using sort_values().

as_index=False: This ensures the 'Product' remains a column in the result, not an index.

The error occurs because in the Plotly px.bar() function, the column names you reference must match exactly with the names in your DataFrame. The error says that 'Total_Sales' is not recognized, but your DataFrame column is named 'Total Sales' (with a space).

Solution:
You need to correct the column name in the y argument to match the column name exactly as it appears in your DataFrame, i.e., 'Total Sales'.

In [ ]:
# Define the correct path to the CSV file
file_path = Path("/home/charles/github/repos/learn-pandas/data/mock_data.csv")

# Load the data
data = pd.read_csv(file_path)

# Assuming 'Total Sales' column exists in data, if not, create it
if (
    "Total Sales" not in data.columns
    and "Sales" in data.columns
    and "Quantity" in data.columns
):
    data["Total Sales"] = data["Sales"] * data["Quantity"]

# Group the data by 'Product' and calculate the total sales for each product
product_sales = data.groupby("Product", as_index=False)["Total Sales"].sum()

# Create a bar chart using Plotly Express
fig = px.bar(
    product_sales, x="Product", y="Total Sales", title="Total Sales by Product"
)  # Note: 'Total Sales' is used here

# Show the chart
fig.show()

In [ ]:
# Define the correct path to the CSV file
file_path = Path("/home/charles/github/repos/learn-pandas/data/mock_data.csv")

# Load the data
data = pd.read_csv(file_path)

# Convert 'Date' column to datetime
data["Date"] = pd.to_datetime(data["Date"])

# Filter data for a specific time range
filtered_data = data[
    (data["Date"] > "2019-03-22") & (data["Date"] < "2019-10-26")
].copy()

# Create 'Total Sales' column in filtered_data if it doesn't exist
if (
    "Total Sales" not in filtered_data.columns
    and "Sales" in filtered_data.columns
    and "Quantity" in filtered_data.columns
):
    filtered_data["Total Sales"] = filtered_data["Sales"] * filtered_data["Quantity"]

# Group data by month
filtered_data.set_index("Date", inplace=True)  # Set 'Date' as index to use resample()
monthly_sales = (
    filtered_data.resample("ME")["Total Sales"].sum().reset_index()
)  # 'ME' for month-end frequency

# Create a line chart using Plotly Express
fig = px.line(monthly_sales, x="Date", y="Total Sales", title="Monthly Sales Trend")

# Show the chart
fig.show()